# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_deterministic.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data="collected_data/rl_purestochastic.txt")
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([i for i in range(2000)])

start
[ 0.00000000e+00  7.95731469e+08  1.14110892e-01  2.46000047e-02
 -2.40000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01 -6.00000000e-01]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.33891077e-02 -4.23999953e-02
  2.84999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.95938047e-01 -6.00000000e-01 -4.85130982e-02]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.97089108e-01 -5.19999953e-02
 -1.57000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.84874650e-01  5.98976918e-01 -3.12414656e-01]
Read chunk # 3 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.33189108e-01  1.48000047e-02
  4.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.33187685e-01  5.18887883e-01  2.92787205e-01]
Read chunk # 4 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.58910892e-01 -5.15999953e-02
  1.93999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000

start
[ 0.00000000e+00  7.95731469e+08 -1.30089108e-01  1.26000047e-02
  2.65999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -3.00498971e-01 -1.09507040e-01]
Read chunk # 44 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.35910892e-01 -5.37999953e-02
 -2.50000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.38838169e-01  6.00000000e-01 -3.75898719e-01]
Read chunk # 45 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.08910771e-03  5.96000047e-02
 -2.94000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01 -6.00000000e-01]
Read chunk # 46 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.03810892e-01  9.20000469e-03
 -2.18000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -4.59576187e-01 -6.00000000e-01]
Read chunk # 47 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -5.44891077e-02 -4.83999953e-02
 -2.32000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.2198

start
[ 0.00000000e+00  7.95731469e+08  6.52108923e-02 -3.41999953e-02
 -3.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.91108872e-01  6.00000000e-01  6.00000000e-01]
Read chunk # 90 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.33910892e-01  3.22000047e-02
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -8.45803493e-02  1.26420156e-01  3.84138893e-01]
Read chunk # 91 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.13889108e-01 -4.77999953e-02
 -6.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.37698138e-01 -5.84863104e-01  1.26168065e-01]
Read chunk # 92 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.03310892e-01 -3.81999953e-02
 -2.60000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01 -2.81368998e-01]
Read chunk # 93 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.46510892e-01 -4.51999953e-02
  3.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.4518

[ 0.00000000e+00  7.95731469e+08  4.03010892e-01 -5.37999953e-02
 -1.47000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.90620116e-01  5.40034474e-01  6.00000000e-01]
Read chunk # 138 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.86810892e-01  7.40000469e-03
 -4.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -1.77450742e-01  5.93783869e-01]
Read chunk # 139 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.99110892e-01 -4.99999953e-02
 -2.75000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01 -6.00000000e-01]
Read chunk # 140 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.46610892e-01 -3.79999953e-02
  2.20999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.58817901e-01  3.59906503e-02  6.00000000e-01]
Read chunk # 141 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.09289108e-01 -5.73999953e-02
  4.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.000000

[ 0.00000000e+00  7.95731469e+08 -3.65889108e-01  3.06000047e-02
 -2.15000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.97465868e-01 -4.70555188e-01 -1.77657165e-01]
Read chunk # 187 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.33689108e-01 -1.83999953e-02
 -1.80000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -1.84808444e-01  5.59598370e-01]
Read chunk # 188 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.98489108e-01  7.20000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.75998508e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 189 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.78110892e-01 -4.47999953e-02
 -2.81000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  1.96648664e-02  6.00000000e-01]
Read chunk # 190 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.99108923e-02  5.14000047e-02
 -1.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.310905

...

[ 0.00000000e+00  7.95731469e+08 -4.32189108e-01 -4.29999953e-02
  4.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.09513570e-01  1.84095824e-01  1.85422746e-01]
Read chunk # 1811 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.87210892e-01  2.52000047e-02
 -2.07000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.12152461e-01 -5.49303658e-01 -9.56883855e-02]
Read chunk # 1812 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.69108923e-02 -2.77999953e-02
  1.90999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.43848681e-02  6.00000000e-01  6.00000000e-01]
Read chunk # 1813 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -6.16891077e-02  1.58000047e-02
 -8.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  3.89156433e-01]
Read chunk # 1814 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.90189108e-01 -2.03999953e-02
 -9.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.82

[ 0.00000000e+00  7.95731469e+08  4.20710892e-01 -2.77999953e-02
 -2.33000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.09539286e-01  2.88121640e-01  3.88875427e-01]
Read chunk # 1856 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.02010892e-01 -8.39999531e-03
  2.68999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.14359323e-01  5.64374331e-01  1.22091660e-01]
Read chunk # 1857 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.21989108e-01  8.60000469e-03
 -1.83000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -1.59982083e-01 -1.47306429e-01]
Read chunk # 1858 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.70310892e-01  2.66000047e-02
  2.04999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.39006116e-01 -2.41762819e-01  4.57396317e-01]
Read chunk # 1859 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.96689108e-01  5.82000047e-02
  2.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00

[ 0.00000000e+00  7.95731469e+08  7.61089229e-03  1.36000047e-02
 -6.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.45514124e-01  6.00000000e-01 -3.45621909e-01]
Read chunk # 1904 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.15410892e-01 -4.67999953e-02
 -1.43000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.28180442e-01 -6.00000000e-01 -5.91834629e-01]
Read chunk # 1905 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.57910892e-01  4.18000047e-02
  1.89998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.17487952e-01  4.37245698e-01 -6.00000000e-01]
Read chunk # 1906 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.17089108e-01 -4.19999953e-02
  7.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -2.86216320e-01  6.00000000e-01]
Read chunk # 1907 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  6.25108923e-02 -2.13999953e-02
 -1.17000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00

[ 0.00000000e+00  7.95731469e+08  1.96310892e-01  3.42000047e-02
 -5.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.62048999e-01  5.66433524e-02 -6.00000000e-01]
Read chunk # 1948 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.35710892e-01 -4.19999953e-02
  7.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.72990701e-02 -3.66037139e-02 -1.42005470e-01]
Read chunk # 1949 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.59410892e-01  3.20000469e-03
  6.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.64576497e-01 -6.00000000e-01  5.86511624e-01]
Read chunk # 1950 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.82710892e-01  1.60000047e-02
 -2.91000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -4.75406591e-01 -6.00000000e-01]
Read chunk # 1951 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.18089108e-01  7.40000469e-03
  1.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.87

[ 0.00000000e+00  7.95731469e+08  3.90410892e-01  1.24000047e-02
  9.89998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  5.24360876e-01]
Read chunk # 1989 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.48610892e-01 -3.87999953e-02
  1.98999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01 -6.00000000e-01]
Read chunk # 1990 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.99108923e-02  1.28000047e-02
  3.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.44419055e-01  5.87838278e-01 -8.12496758e-03]
Read chunk # 1991 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.05589108e-01 -1.51999953e-02
  2.33999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.56807043e-01  6.00000000e-01  6.00000000e-01]
Read chunk # 1992 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.45689108e-01 -1.59999531e-03
  7.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.37

In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -286.53177,
 'std': 142.0723,
 'min': -662.35175,
 'max': 0.0,
 'histogram': (array([ 11,  29,  40,  71,  63,  71,  75,  97, 106, 123, 145, 177, 178,
         166, 167, 188, 158, 116,  17,   1]),
  array([-662.35175 , -629.23413 , -596.1166  , -562.99896 , -529.8814  ,
         -496.7638  , -463.6462  , -430.52863 , -397.41104 , -364.29346 ,
         -331.17587 , -298.0583  , -264.9407  , -231.8231  , -198.70552 ,
         -165.58794 , -132.47035 ,  -99.35276 ,  -66.235176,  -33.117588,
            0.      ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [6]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

# Using hyperparameters found from the random search
# [0.004343795776698955, 3.716957428808228e-05, 3]
actor_lr_this_iter = 0.004343795776698955
critic_lr_this_iter = 3.716957428808228e-05
n_steps_this_iter = 3

actor_encoder = VectorEncoderFactory(hidden_units=[12, 24, 36, 24, 12],
                                      activation='relu', use_batch_norm=True, dropout_rate=0.2)
critic_encoder = VectorEncoderFactory(hidden_units=[12, 24, 24, 12],
                                      activation='relu', use_batch_norm=True, dropout_rate=0.2)


model = TD3PlusBC(q_func_factory='qr', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            actor_encoder_factory = actor_encoder,
            critic_encoder_factory = critic_encoder,
            action_scaler=action_scaler,
            actor_learning_rate=actor_lr_this_iter, 
            critic_learning_rate=critic_lr_this_iter,
            n_steps=n_steps_this_iter, 
            use_gpu=True) #change it to true if you have one
model.build_with_dataset(dataset)

In [7]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.010758407734730911


In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [9]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=40, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-20 15:28.06 [debug    ] RoundIterator is selected.
2022-04-20 15:28.06 [info     ] Directory is created at d3rlpy_logs/CQL_20220420152806
2022-04-20 15:28.06 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-20 15:28.06 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-20 15:28.06 [warning  ] Skip building models since they're already built.
2022-04-20 15:28.06 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220420152806/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-20 15:38.14 [info     ] CQL_20220420152806: epoch=1 step=6928 epoch=1 metrics={'time_sample_batch': 0.0003016272400598328, 'time_algorithm_update': 0.08602012817771551, 'temp_loss': 3.252458093502918, 'temp': 0.7384563273477086, 'alpha_loss': -14.509580498617185, 'alpha': 1.4357602483061664, 'critic_loss': 22.86941206895727, 'actor_loss': 39.18387807384648, 'time_step': 0.08663777600948035, 'td_error': 15.165782817228687, 'init_value': -82.5303726196289, 'ave_value': -83.63484396477433} step=6928
2022-04-20 15:38.14 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420152806/model_6928.pt


Epoch 2/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-20 15:48.05 [info     ] CQL_20220420152806: epoch=2 step=13856 epoch=2 metrics={'time_sample_batch': 0.0003192173897807097, 'time_algorithm_update': 0.0833366665705247, 'temp_loss': 0.7667452684515177, 'temp': 0.42597445298425046, 'alpha_loss': -13.523500026709144, 'alpha': 2.8465004737637334, 'critic_loss': 54.11067065066609, 'actor_loss': 117.59847816749203, 'time_step': 0.08401468031400897, 'td_error': 35.829020571729394, 'init_value': -158.5977325439453, 'ave_value': -160.53637813901472} step=13856
2022-04-20 15:48.05 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420152806/model_13856.pt


...

Epoch 39/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-20 21:58.32 [info     ] CQL_20220420152806: epoch=39 step=270192 epoch=39 metrics={'time_sample_batch': 0.0003448767075637747, 'time_algorithm_update': 0.0929982915830392, 'temp_loss': -0.00018753714137737516, 'temp': 0.3766378736334227, 'alpha_loss': -0.011947945028642411, 'alpha': 2.314976746160494, 'critic_loss': 285.85863863396315, 'actor_loss': 371.0312901600404, 'time_step': 0.09372722927059367, 'td_error': 141.4734840362178, 'init_value': -373.0046081542969, 'ave_value': -379.5004974193401} step=270192
2022-04-20 21:58.32 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420152806/model_270192.pt


Epoch 40/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-20 22:09.35 [info     ] CQL_20220420152806: epoch=40 step=277120 epoch=40 metrics={'time_sample_batch': 0.0003421332420432815, 'time_algorithm_update': 0.09344303563202777, 'temp_loss': 0.0029535965536136017, 'temp': 0.375632077773848, 'alpha_loss': 0.02483469982435978, 'alpha': 2.30095637584531, 'critic_loss': 284.19343940758813, 'actor_loss': 370.460908828101, 'time_step': 0.09416803579958151, 'td_error': 141.2749599119681, 'init_value': -372.5274963378906, 'ave_value': -378.97610149782196} step=277120
2022-04-20 22:09.35 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420152806/model_277120.pt


[(1,
  {'time_sample_batch': 0.0003016272400598328,
   'time_algorithm_update': 0.08602012817771551,
   'temp_loss': 3.252458093502918,
   'temp': 0.7384563273477086,
   'alpha_loss': -14.509580498617185,
   'alpha': 1.4357602483061664,
   'critic_loss': 22.86941206895727,
   'actor_loss': 39.18387807384648,
   'time_step': 0.08663777600948035,
   'td_error': 15.165782817228687,
   'init_value': -82.5303726196289,
   'ave_value': -83.63484396477433}),
 (2,
  {'time_sample_batch': 0.0003192173897807097,
   'time_algorithm_update': 0.0833366665705247,
   'temp_loss': 0.7667452684515177,
   'temp': 0.42597445298425046,
   'alpha_loss': -13.523500026709144,
   'alpha': 2.8465004737637334,
   'critic_loss': 54.11067065066609,
   'actor_loss': 117.59847816749203,
   'time_step': 0.08401468031400897,
   'td_error': 35.829020571729394,
   'init_value': -158.5977325439453,
   'ave_value': -160.53637813901472}),
 (3,
  {'time_sample_batch': 0.0003262850483900956,
   'time_algorithm_update': 0.08

In [15]:
model.save_model('cqlPurestoch2000_Ep40model_CPUOnly.pt')
model.save_policy('cqlPurestoch2000_Ep40_CPUOnly.pt')

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [11]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i+2000 for i in range(100)]) #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [12]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_stochastic.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [13]:
# from d3rlpy.torch_utility import to_cpu
# to_cpu(model)
# model.save_policy("cqlStochpid2000Ep40CPU.pt")
# model.save_model("cqlStochpid2000Ep40modelCPU.pt")

In [14]:
# for key in dir(model):
#     module = getattr(model, key)
#     if isinstance(module, (torch.nn.Module, torch.nn.Parameter)):
#         print(yes)
#         print(key)
# dir(model)
# type(model)
# model.cpu()
# from d3rlpy.algos.torch.base import TorchImplBase
# new_model = TorchImplBase()
# from d3rlpy.torch_utility import _get_attributes
# model._device = "cpu:0"
# print(model._device)


# def my_get_state_dict(impl: Any) -> Dict[str, Any]:
#     rets = {}
#     for key in _get_attributes(impl):
#         obj = getattr(impl, key)
#         if isinstance(obj, (torch.nn.Module, torch.optim.Optimizer)):
#             if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#                 obj.cpu()
#             rets[key] = obj.state_dict()
#     return rets

# torch.save(my_get_state_dict(model), "my_test_model.pt")

# for key in dir(model):
#     obj = getattr(model, key)
#     if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#         obj.cpu()
#         print("convert to cpu")
# model.save_policy("cqlStochpid2000Ep40modelCPU.pt")

# import trace
# tracer = trace.Trace()
# tracer.run('model.save_policy("cqlStochpid2000Ep40modelCPU.pt")')